<a href="https://colab.research.google.com/github/samuelcmaggi/assistente-voz-python/blob/main/assistente_voz_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1: Instalações (Rode isso primeiro)

In [10]:
!pip install git+https://github.com/openai/whisper.git -q
!pip install openai gTTS
!sudo apt-get install -y ffmpeg

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


2: Funções de Gravação

In [11]:
# Importações
import whisper
import os
import openai
from gtts import gTTS
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

# Configuração da API Key (Troque pelo seu, ou use secrets do Colab)
# Se for usar direto, cuidado para não vazar sua chave no GitHub!
# O ideal é usar: from google.colab import userdata; api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = 'from getpass import getpass
minha_api_key = getpass("Digite sua API Key: ")'
openai.api_key = os.environ.get('OPENAI_API_KEY')

# Inicializa o cliente OpenAI
client = openai.OpenAI()

# --- Função para gravar áudio no navegador ---
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  display(Javascript(RECORD))
  js_result = output.eval_js('record(%s)' % (sec * 1000))
  audio = b64decode(js_result.split(',')[1])
  file_name = 'request_audio.wav'
  with open(file_name, 'wb') as f:
    f.write(audio)
  return f'/content/{file_name}'

print("Configuração concluída!")

Configuração concluída!


3: Executar o Assistente

In [12]:
# --- IMPORTANTE: Instale a biblioteca antes se ainda não tiver feito ---
# !pip install openai

import os
from openai import OpenAI
import whisper
# Se precisar de outras importações (como gTTS, record, etc), mantenha as que você já tinha nas células anteriores

# 1. Configuração do Cliente OpenAI
# COLE SUA CHAVE DENTRO DAS ASPAS ABAIXO:
minha_api_key = "from getpass import getpass
minha_api_key = getpass("Digite sua API Key: ")"

client = OpenAI(api_key=minha_api_key)

# ---------------------------------------------------------
# O RESTANTE DO SEU CÓDIGO (JÁ INTEGRADO):

# ... (Seu código de gravação de áudio deve vir antes aqui) ...
# Vou assumir que você já gravou e tem a variável 'record_file' pronta, como na sua imagem.

# Definição do idioma
language = 'pt'

# 2. Transcrever (Whisper)
print('Transcrevendo...')
# Carrega o modelo (pode demorar um pouquinho na primeira vez)
model = whisper.load_model("small")
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result["text"]
print(f"Você disse: {transcription}")

# 3. Enviar para a IA (ChatGPT)
if transcription.strip():
    print('Consultando a IA...')
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Você é um assistente útil."},
                {"role": "user", "content": transcription}
            ]
        )
        chatgpt_response = response.choices[0].message.content
        print(f"Resposta: {chatgpt_response}")

        # 4. Falar a Resposta (gTTS)
        # (Certifique-se de ter importado gTTS e Audio lá em cima ou na célula anterior)
        print('Gerando áudio...')
        from gtts import gTTS
        from IPython.display import Audio, display

        gtts_object = gTTS(text=chatgpt_response, lang=language, slow=False)
        response_audio = "response_audio.wav"
        gtts_object.save(response_audio)
        display(Audio(response_audio, autoplay=True))

    except Exception as e:
        print(f"Erro ao conectar com a OpenAI: {e}")

else:
    print("Não consegui ouvir nada.")

Transcrevendo...
Você disse:  Se vocês beficam, a paz à vida per нравится
Consultando a IA...
Erro ao conectar com a OpenAI: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
